In [1]:
!pip install -q scanpy anndata

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import scanpy as sc
import anndata
import scipy
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [45]:
meta = pd.read_csv('https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE190604&format=file&file=GSE190604%5Fcellranger%2Dguidecalls%2Daggregated%2Dunfiltered%2Etxt%2Egz', compression = 'gzip', sep = '\t')
meta.rename(columns = {'cell_barcode':'CellID'}, inplace = True)
meta.head()

,CellID,num_features,feature_call,num_umis
0,GGGAGATAGACCGTTT-1,1,ABCB10-1,216
1,GACGCTGCATTGTCGA-1,1,ABCB10-1,149
2,TTAATCCTCGTGCACG-1,1,ABCB10-1,170
3,ACACGCGTCGACCTAA-1,1,ABCB10-1,131
4,CATCCACCATCGATGT-1,1,ABCB10-1,64


In [40]:
features = pd.read_csv('https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE190604&format=file&file=GSE190604%5Ffeatures%2Etsv%2Egz', compression = 'gzip', sep = '\t', header = None)
features.drop(columns = [0], inplace = True)
features.rename(columns = {1:'Genes', 2:'mRNA_CRISPR_guide'}, inplace = True)
features.index = features['Genes']
features.index.name = None
features.head()

,Genes,mRNA_CRISPR_guide
MIR1302-2HG,MIR1302-2HG,Gene Expression
FAM138A,FAM138A,Gene Expression
OR4F5,OR4F5,Gene Expression
AL627309.1,AL627309.1,Gene Expression
AL627309.3,AL627309.3,Gene Expression


In [32]:
barcodes = pd.read_csv('https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE190604&format=file&file=GSE190604%5Fbarcodes%2Etsv%2Egz', compression = 'gzip', sep = '\t', header = None)
barcodes.rename({0:'CellID'}, inplace = True, axis = 1)
barcodes.head()

,CellID
0,AAACCCACAACAAGAT-1
1,AAACCCACAACGGCTC-1
2,AAACCCACACAGAAGC-1
3,AAACCCACACCCTGTT-1
4,AAACCCACACTATGTG-1


In [24]:
!curl 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE190604&format=file&file=GSE190604%5Fmatrix%2Emtx%2Egz' -o matrix.mtx.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  950M  100  950M    0     0  92.9M      0  0:00:10  0:00:10 --:--:-- 67.0M


In [25]:
matrix = scipy.io.mmread('matrix.mtx.gz')
matrix = matrix.transpose()
matrix = scipy.sparse.csr_matrix(matrix)

In [41]:
adata = anndata.AnnData(X = matrix, var = features, obs = barcodes)
adata

/usr/local/lib/python3.10/dist-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 103805 × 36755
    obs: 'CellID'
    var: 'Genes', 'mRNA_CRISPR_guide'